<h1>Download data</h1>

In [ ]:
from src.download import download

# Your data will be downloaded to the data folder
download()

<h1>Model</h1>

In [ ]:
import json
import pandas as pd
from src.load_data import load_data
from src.preprocessing import data_preprocessing
from src.model import Model


stimuli = "MVC"
# stimuli = "SVC"

data = load_data(f"{stimuli}.tsv")
data = data_preprocessing(data)
model = Model(data)
results = model.cross_validation(5, 'ZR', 'DT', 'KNN', 'NB', 'SVM', 'LR', 'AB', 'RF')
# results = model.leave_one_session_out_cross_validation('ZR', 'DT', 'KNN', 'NB', 'SVM', 'LR', 'AB', 'RF')

result_df = pd.DataFrame(results)
result_df